<img src= "img/pipelines.png" style="height:450px">


[Image Source](https://towardsdatascience.com/using-functiontransformer-and-pipeline-in-sklearn-to-predict-chardonnay-ratings-9b13fdd6c6fd)

__Agenda__

- Pipelines and Composite estimators

- Why do we need them?

- How to use them in sklearn: accessing a particular object in pipe and changing parameters

- ROC Curve

- Area under ROC Curve

In [ ]:
import pandas as pd
import numpy as np

# Pipelines

__What is a Pipeline?__

_Transformers:_ Any object with .transform method. Ex: StandardScaler, OneHotEncoder.

_Estimators:_ Any object with predict method. Ex: RandomForestClassifier, LinearRegression etc.

_Pipelines:_ A tool for combining transformers with estimators. 


__Why do we need pipelines?__

- Convenience and encapsulation

Even though we train 10 transformers and 5 estimators we will call fit and predict once.

- Joint parameter selection - here emphasize preprocessing part

We can put pipelines into gridsearch and find best parameters for all the estimators at once.

- Safety

Pipelines help avoid leaking statistics from your test data into trained model.

[Pipelines and Composite Estimators](https://scikit-learn.org/stable/modules/compose.html#combining-estimators)



## Usage of Pipelines

The Pipeline is built using a list of (key, value) pairs, where the key is a string containing the name you want to give this step and value is an estimator object:

In [ ]:
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression

estimators = [('imputer', SimpleImputer()),
              ('scaler', StandardScaler()),
              ('clf', LogisticRegression())]

pipe = Pipeline(estimators)

pipe

__Your Turn__

- Create your own pipeline. You can use the same transformers and estimators with different parameters and 'name'.

- You can create a new pipe with a scaler also.

In [ ]:
your_pipe = None

Sklearn also gives us "make_pipeline" which is almost the same thing but with make_pipeline you don't have to give names.

__Your Turn__

-  [Check documentation: 6.1.1.1.1 Construction](https://scikit-learn.org/stable/modules/compose.html) and use `make_pipeline` to construct a pipeline.

In [ ]:
from sklearn.pipeline import make_pipeline

In [ ]:
your_make_pipe = None

## Accessing steps

We have multiple ways to access and object in the pipeline

In [ ]:
## note that these will all give the simple imputer object
pipe[0]

pipe['imputer']

pipe.steps[0][1]

In [ ]:
## We can also access a particular object by named_steps

pipe.named_steps.imputer

In [ ]:
## We can 'slice' pipelines to create sub-pipes

pipe[1:]

## Access to the parameters

Parameters of the estimators in the pipeline can be accessed using the 
"estimator__parameter" syntax.

In [ ]:
pipe.set_params(clf__C = 10)

pipe

## Pipelines in action

In [ ]:
from sklearn.model_selection import train_test_split

[Dataset info](https://archive.ics.uci.edu/ml/datasets/Heart+Disease)

In [ ]:
df = pd.read_csv('./data/heart.csv')

In [ ]:
print(df.shape)
df.head(3)

In [2]:
!open .

In our dataset we have 303 patients and 13 independent variables and 1 binary target variable.

When we are working with classification problems it is always good practice to check the class balance.

In [ ]:
df['target'].value_counts(normalize=True)

We see that approximately 54% of the patients are in the class 0 which refers to 'no presence' of a heart disease. Consequently, 45% of the patients have a heart disease. 

## Creating Train-Test Split

In [ ]:
# For model evaluation we split our data into two parts: Train - Test

X = df.drop('target', axis=1)
y = df['target']
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=77, stratify=y,
                                                    test_size=.2)

In [ ]:
# Let's check number of 1 and 0 in both datasets
y_train.mean(), y_test.mean()

## Choosing a Perfomance Metric for Model Evaluations

__Model Selection vs Model Evaluation__

- Model Selection/Model Comparison: What is the best parameters for a given model. Between different models which one is better models the reality.

Ex: If we are working with an app that runs a machine learning algorithm model selection is choosing the process of choosing a final algorithm to deploy.


- Model Evaluation: After selecting a 'best' model with model selection how this model will perform in the 'real' case.

Ex: Model evaluation is where we want to predict how successful this algorithm will be.

[Available tools in sklearn](https://scikit-learn.org/stable/model_selection.html)

<img src= 'img/table.png' width = 450 />

## Data Prep Before Training a Model

[A good blog post on handling categorical variables](https://www.bogotobogo.com/python/scikit-learn/scikit_machine_learning_Data_Preprocessing-Missing-Data-Categorical-Data.php)

__Your Turn__

- Convert Categorical Variables to OneHotEncoding


In [ ]:
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer

In [ ]:
ss = StandardScaler()

In [ ]:
# create an encoder object. This will help us to convert
# categorical variables to new columns
encoder = None

# Create an columntransformer object.
# This will help us to merge transformed columns
# with the rest of the dataset.
catvars = None
ct = ColumnTransformer(???)



__Don't forget!!__

- Apply the same transformations to the test data.

In [ ]:
Xtest = ct.transform(X_test)
Xtest.shape

__Scaling Features__ 

-- Let's go back to the column transformer.

[Different Scalers and Their Effect on Data](https://scikit-learn.org/stable/auto_examples/preprocessing/plot_all_scaling.html#sphx-glr-auto-examples-preprocessing-plot-all-scaling-py)

In [ ]:
from sklearn.preprocessing import StandardScaler

In [ ]:
X_test.shape

In [ ]:
np.mean(X, axis=0)

# What do you expect if you check the means of X_test? Try

## Model Training

[Check sklearn for documentation of Logistic Regression](https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.LogisticRegression.html)


[For solvers](https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression)

In [ ]:
log_reg = LogisticRegression(penalty='none', max_iter=10000)
log_reg.fit(X, y_train)

In [ ]:
# What is this score?
print(log_reg.score(X, y_train))

## Confusion Matrix

In [ ]:
from sklearn.metrics import confusion_matrix
import seaborn as sns

In [ ]:
y_pred = log_reg.predict(X)

score = log_reg.score(X, y_train)


In [ ]:
cm = confusion_matrix(y_train, y_pred)

In [ ]:
plt.figure(figsize=(10, 10))
sns.heatmap(cm, annot=True, fmt=".0f", linewidths=.5,
            square=True, cmap='Blues')
plt.ylabel('Actual label')
plt.xlabel('Predicted label')
all_sample_title = 'Accuracy Score: {0}'.format(score)
plt.title(all_sample_title, size=15)
plt.savefig('toy_Digits_ConfusionSeabornCodementor.png')
# plt.show();

## ROC Curves for Model Selection

### Plotting ROC curves

In [ ]:
log_reg_vanilla = LogisticRegression(penalty='none', max_iter=10000)

log_reg_l2 = LogisticRegression(penalty='l2', C=0.01, max_iter=10000)

In [ ]:
log_reg_vanilla.fit(X, y_train)

y_probs_vanilla = log_reg_vanilla.predict_proba(X)

In [ ]:
log_reg_l2.fit(X, y_train)
y_probs_l2 = log_reg_l2.predict_proba(X)

In [ ]:
import matplotlib.pyplot as plt

%matplotlib inline

In [ ]:
from sklearn.metrics import roc_curve

In [ ]:
fpr_v, tpr_v, thresholds_v = roc_curve(y_train, y_probs_vanilla[:,1])
fpr_l2, tpr_l2, thresholds_l2 = roc_curve(y_train, y_probs_l2[:,1])

In [ ]:
def plot_roc_curve(fpr, tpr, label=None):
    plt.plot(fpr, tpr, linewidth=2, label=label)
    plt.plot([0, 1], [0, 1], 'k--')
    plt.axis([0, 1, 0, 1])
    plt.xlabel('False Positive Rate')
    plt.ylabel('True Positive Rate')


plot_roc_curve(fpr_v, tpr_v, label='Vanilla')
plot_roc_curve(fpr_l2, tpr_l2, label='L2-Penalty')
plt.legend()
plt.show()

Also we can measure the __A__rea __U__nder __C__urve scores


In [ ]:
from sklearn.metrics import roc_auc_score

In [ ]:

roc_auc_score(y_train, y_probs_vanilla[:,1])

In [ ]:
roc_auc_score(y_train, y_probs_l2[:,1])

### The Default Measure (in most prebuilt models) - Accuracy

$$ \frac{(TP + TN)}{(TP + FP + TN + FN)} $$

Category definitions - possible outcomes in binary classification

- TP = True Positive (class 1 correctly classified as class 1) - e.g. Patient with cancer tests positive for cancer
- TN = True Negative (class 0 correctly classified as class 0) - e.g. Patient without cancer tests negative for cancer
- FP = False Positive (class 0 incorrectly classified as class 1) - e.g. Patient without cancer tests positive for cancer
- FN = False Negative (class 1 incorrectly classified as class 0) - e.g. Patient with cancer tests negative for cancer

 $$ \text{Possible misclassifications} $$

<img src='./img/type-1-type-2.jpg' width=400/>
 

Remember that Logistic Regression gives probability predictions for each class, in addition to the final classification. By default, threshold for the prediction is set to 0.5, but we can adjust that threshold.

### The AUC / ROC curve (Area Under Curve of the Receiver Operating Characteristic)

<img src='img/pop-curve.png' width=500/>


### Using Cross Validation Scores for Model Evaluation

In [ ]:
from sklearn.model_selection import cross_val_score

In [ ]:
log_reg = LogisticRegression(penalty='none', max_iter=10000)

In [ ]:
y_scores = cross_val_score(log_reg, X, y_train, cv=3, scoring='roc_auc')

In [ ]:
y_scores